<a href="https://colab.research.google.com/github/BookHaven12/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimization 1- Use Keras Tuner to determine the optimal amount of layers and neurons

## Preprocessing

In [58]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [59]:
# Drop columns 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [60]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [61]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [62]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [63]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C1248,1
C6100,1
C1820,1


In [64]:
# Look at CLASSIFICATION value counts >1
counts = application_df['CLASSIFICATION'].value_counts()
filtered_counts = counts[counts>1]
filtered_counts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [65]:
# Choose a cutoff value and create a list of classifications to be replaced
# .index filters down to the classification names to replace (creating a list of classification names that appear less than 1000 times in the data)
classifications_to_replace = counts[counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [66]:
# Convert categorical data to numeric with `pd.get_dummies`
classification_dummies = pd.get_dummies(application_df)

In [67]:
# Split our preprocessed data into our features and target arrays
y = classification_dummies["IS_SUCCESSFUL"]
X = classification_dummies.drop(columns="IS_SUCCESSFUL", axis="columns").values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [68]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [69]:
classification_dummies.shape[1]

44

## Compile, Train and Evaluate the Model

In [16]:
# This counts how many features (columns) your data has, so the model knows what kind of input to expect.
number_input_features = len(X_train[0])

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','selu'])

    # Allow kerastuner to decide number of neurons in first layer
    nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=16,
        max_value=130,
        step=8), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=4,
            max_value=64,
            step=8),
            activation=activation))

    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn

In [17]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 7.0 MB/s eta 0:00:00


In [18]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 04m 02s]
val_accuracy: 0.7316617965698242

Best val_accuracy So Far: 0.7336443066596985
Total elapsed time: 02h 44m 04s


In [21]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 88,
 'num_layers': 5,
 'units_0': 28,
 'units_1': 44,
 'units_2': 52,
 'units_3': 20,
 'units_4': 20,
 'units_5': 28,
 'tuner/epochs': 17,
 'tuner/initial_epoch': 6,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0156'}

In [22]:
# # Evaluate best model against full test data
# best_model = tuner.get_best_models(1)[0]
# model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the model to a file
best_model.save('AlphabetSoupCharity_Optimization_test_1')

268/268 - 1s - 4ms/step - accuracy: 0.7336 - loss: 0.5531
Loss: 0.5531476140022278, Accuracy: 0.7336443066596985


# Optimization Test #2 - Drop 'ASK_AMT' and use the optimized layers and neurons from the previous test


### Preprocessing for Model 2

In [ ]:
# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Drop 'EIN', 'NAME', and 'ASK_AMT' columns
application_df = application_df.drop(columns = ['EIN', 'NAME', 'ASK_AMT'])

# Create a list of application types to be replaced
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

# Look at CLASSIFICATION value counts >1
counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced
# .index filters down to the classification names to replace (creating a list of classification names that appear less than 1000 times in the data)
classifications_to_replace = counts[counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
classification_dummies = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
y = classification_dummies["IS_SUCCESSFUL"]
X = classification_dummies.drop(columns="IS_SUCCESSFUL", axis="columns").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate Model 2

In [ ]:
# Create the Model
# This counts how many features (columns) your data has, so the model knows what kind of input to expect.
number_input_features = len(X_train[0])

hidden_nodes_layer1 = 88
hidden_nodes_layer2 = 28
hidden_nodes_layer3 = 44
hidden_nodes_layer4 = 52
hidden_nodes_layer5 = 20
hidden_nodes_layer6 = 20

# Start building the Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim= number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer2, activation="relu"))

# Third Hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer3, activation="relu"))

# Fourth Hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer4, activation="relu"))

# Fifth Hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer5, activation="relu"))

# Sixth Hidden layer
nn.add(tf.keras.layers.Dense(units= hidden_nodes_layer6, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#Save File
nn.save('AlphabetSoupCharity_Optimization_test_2.keras')

# Optimization Test #3- Change Activiation to LeakyReLu

### Preprocessing for Model 3

In [ ]:
# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Drop 'EIN', 'NAME', and 'ASK_AMT' columns
application_df = application_df.drop(columns = ['EIN', 'NAME', 'ASK_AMT'])

# Create a list of application types to be replaced
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

# Look at CLASSIFICATION value counts >1
counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced
# .index filters down to the classification names to replace (creating a list of classification names that appear less than 1000 times in the data)
classifications_to_replace = counts[counts < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
classification_dummies = pd.get_dummies(application_df)

# Split our preprocessed data into our features and target arrays
y = classification_dummies["IS_SUCCESSFUL"]
X = classification_dummies.drop(columns="IS_SUCCESSFUL", axis="columns").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate Model 3

In [ ]:
from tensorflow.keras.layers import Dense, LeakyReLU

# Count number of input features
number_input_features = len(X_train[0])

# Define hidden layer sizes
hidden_nodes_layer1 = 88
hidden_nodes_layer2 = 28
hidden_nodes_layer3 = 44
hidden_nodes_layer4 = 52
hidden_nodes_layer5 = 20
hidden_nodes_layer6 = 20

# Build the model
nn = tf.keras.models.Sequential()

# Input layer + Leaky ReLU
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features))
nn.add(LeakyReLU(alpha=0.1))

# Hidden layers + Leaky ReLU
nn.add(Dense(units=hidden_nodes_layer2))
nn.add(LeakyReLU(alpha=0.1))

nn.add(Dense(units=hidden_nodes_layer3))
nn.add(LeakyReLU(alpha=0.1))

nn.add(Dense(units=hidden_nodes_layer4))
nn.add(LeakyReLU(alpha=0.1))

nn.add(Dense(units=hidden_nodes_layer5))
nn.add(LeakyReLU(alpha=0.1))

nn.add(Dense(units=hidden_nodes_layer6))
nn.add(LeakyReLU(alpha=0.1))

# Output layer
nn.add(Dense(units=1, activation="sigmoid"))

# Print summary
nn.summary()

# Compile model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#Save File
nn.save('AlphabetSoupCharity_Optimization_test_3.keras')

Model 3 Accuracy:
Model 3 Loss:

# ANALYSIS


In [ ]:
## Test # 1 - removed ASK_AMT Column and tested for LeakyReLu
# from tensorflow.keras.layers import LeakyReLU

# # This counts how many features (columns) your data has, so the model knows what kind of input to expect.
# number_input_features = len(X_train[0])

# def create_model(hp):
#     nn = tf.keras.models.Sequential()

#     activation_choice = hp.Choice('activation', ['relu', 'tanh', 'selu', 'leaky_relu'])

#     # New Input layer (this removes the warning)
#     nn.add(tf.keras.Input(shape=(number_input_features,)))

#     # First hidden layer (no more input_dim needed)
#     nn.add(tf.keras.layers.Dense(
#         units=hp.Int('first_units', min_value=16, max_value=130, step=8)
#     ))

#     # Add activation (handling LeakyReLU separately)
#     if activation_choice == 'leaky_relu':
#         nn.add(tf.keras.layers.LeakyReLU(alpha=0.1))
#     else:
#         nn.add(tf.keras.layers.Activation(activation_choice))

#     # Additional hidden layers
#     for i in range(hp.Int('num_layers', 1, 6)):
#         nn.add(tf.keras.layers.Dense(
#             units=hp.Int(f'units_{i}', min_value=4, max_value=64, step=8)
#         ))
#         if activation_choice == 'leaky_relu':
#             nn.add(tf.keras.layers.LeakyReLU(alpha=0.1))
#         else:
#             nn.add(tf.keras.layers.Activation(activation_choice))

#     nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#     nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     return nn



In [ ]:
# ## Code for Test 2 - Removed ASK_AMT and removed 2 activation functions and increased epochs from 20 to 50
# # This counts how many features (columns) your data has, so the model knows what kind of input to expect.
# number_input_features = len(X_train[0])

# # Create a method that creates a new Sequential model with hyperparameter options
# def create_model(hp):
#     nn = tf.keras.models.Sequential()

#     # Allow kerastuner to decide which activation function to use in hidden layers
#     activation = hp.Choice('activation',['relu','selu'])

#     # Allow kerastuner to decide number of neurons in first layer
#     nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#         min_value=16,
#         max_value=130,
#         step=8), activation=activation, input_dim=number_input_features))

#     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#     for i in range(hp.Int('num_layers', 1, 6)):
#         nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
#             min_value=4,
#             max_value=64,
#             step=8),
#             activation=activation))

#     nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#     # Compile the model
#     nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

#     return nn

# Trial # 3 - used the optimized layers from the previous trial , but increased epochs to 200.
# Trial #4 - reduced epochs to 20
# Trial #5 - add back in 'ASK_AMT'